In [30]:
import requests
import pandas as pd
import json

def search_recording(artist_name, release_title):
    base_url = "https://musicbrainz.org/ws/2/release/"
    params = {
        "query": f"artist:\"{artist_name}\" AND release:\"{release_title}\"",
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"} # Replace with your app info

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status() # Raise an exception for bad status codes
    data = response.json()
    #print(data)
    
    if data and data.get('releases'):
        # Return the MBID of the first matching recording
        return data['releases'][1]['id']
    return None

# Example usage:
artist = "Chico Hamilton"
album = "Peregrinations"
release_mbid = search_recording(artist, album)
print(f"Recording MBID for '{album}' by '{artist}': {release_mbid}")

Recording MBID for 'Peregrinations' by 'Chico Hamilton': acfed48f-2f40-4f70-a025-86ea36f17241


In [31]:
def get_credits_for_recording(release_mbid):
    base_url = f"https://musicbrainz.org/ws/2/release/{release_mbid}"
    params = {
        "inc": "artist-credits+labels+recordings+recording-level-rels+work-rels+work-level-rels+artist-rels + release-groups",  # Include artist relationships
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"}

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()
    #print(data)

    credits = []
    
    if data and data.get('artist-credit'):
       for track in data['media'][0]['tracks']:
            for credit in track['recording']['relations']:
                if credit['attributes'] == []:
                    attributes = 'Not Found'
                else:
                    attributes = credit['attributes']
                credits.append((track['number'], track['title'], credit.get('artist', {}).get('name', 'Not Found'), attributes))
            for credit in track['artist-credit']:
                credits.append((track['number'], track['title'], credit.get('name', 'Not Found'), credit.get('attributes', 'Not Found')))
    #print(credits)

    #dataTest = data['media'][0]['tracks'][0]['recording']['relations'][2]['artist']['name']
    #print(dataTest)

    #print("\n#################################################################################################################################\n")

    #credits = [{"Artist" : credit.get('artist', {}).get('name', 'Not Found'), "Role" : credit['attributes'], "Track Number" : track['number'], "Track Title" : track['title']} for track in data['media'][0]['tracks'] for credit in track['recording']['relations']]
    
    return credits

In [32]:
#Example usage:
if release_mbid:
    albumCredits = get_credits_for_recording(release_mbid)
    #print(albumCredits)
else:
    print(f"Recording for '{album}' by '{artist}' not found.")

# Create the DataFrame
df = pd.DataFrame(albumCredits, columns = ['Track Number', 'Track Title', 'Artist', 'Role'])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Display the DataFrame
df

,Track Number,Track Title,Artist,Role
0,1,V-O,Baker Bigsby,Not Found
1,1,V-O,Keg Johnson,Not Found
2,1,V-O,Chico Hamilton,Not Found
3,2,The Morning Side of Love,Baker Bigsby,Not Found
4,2,The Morning Side of Love,Keg Johnson,Not Found
5,2,The Morning Side of Love,Chico Hamilton,Not Found
6,3,Abdullah and Abraham,Baker Bigsby,Not Found
7,3,Abdullah and Abraham,Keg Johnson,Not Found
8,3,Abdullah and Abraham,Chico Hamilton,Not Found
9,4,Andy's Walk,Baker Bigsby,Not Found
